# CONSTRUCTION DE LA BASE DE DONNEES 

- Scraping paroles.net
- API Deezer

In [38]:
import requests
import json
import unidecode
import re
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime
import numpy as np
from langdetect import detect

### Scraping paroles.net : 

In [3]:
def parse_url_lyrics(url):
    """Fonction qui permet de d'ajouter la base de l'url si manquant"""
    base = "https://www.paroles.net"
    if re.search(f"^{base}", url) is None:
        url = base + url
    return url


def get_list_song_artist(artist):
    """pour un artiste renvoie l'ensemble de ses plus grands succès d'après le site paroles.net
    pour chaque chanson, on récupère le lien vers les paroles
    """
    artist = artist.lower()
    base_url = f"https://www.paroles.net/{artist}"
    get_url = requests.get(base_url)
    if get_url.status_code == 200:
        result_html = soup(get_url.text, 'html.parser')
        try:
            # songs = result_html.find('div', {'class': 'song-listing-extra', 'itemprop': 'tracks'}).find(
            # 'div', {'class': f'center-on-mobile best-songs box-content {section}'})
            # Cas 1
            songs = result_html.find(
                'div', {'class': 'song-listing-extra', 'itemprop': 'tracks'})
            list_songs = songs.findAll('a')
            dict_chansons = [{"artist": artist, "title": song.get_text(),
                              "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
            # return dict_chansons
        except Exception:
            try:
                dict_chansons = []
                # Cas 2
                # Autre liste de chansons ?
                for best_song in [' best-songs', '']:
                    for section in ['left', 'right']:
                        list_songs = [i.findAll("a") for i in result_html.findAll(
                            'div', {'class': f'center-on-mobile{best_song} box-content {section}'})][0]
                        if len(list_songs) > 1:
                            dict_chansons = dict_chansons + [{"artist": artist, "title": song.get_text(),
                                                              "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
            except Exception:
                print(f"{artist} - Erreur dans le scraping de la liste des chansons")
                return None

        for section in ['left', 'right']:
            list_songs = [i.findAll("a") for i in result_html.findAll(
                'div', {'class': f'center-on-mobile box-content {section}'})]
            if len(list_songs) == 1 :
                list_songs = list_songs[0]
                if len(list_songs) > 1:
                    dict_chansons = dict_chansons + [{"artist": artist, "title": song.get_text(),
                                                  "url_lyrics": parse_url_lyrics(song['href'])} for song in list_songs]
        return dict_chansons

    else:
        print(f"ERROR pour l'url {base_url}")
        return None


def get_lyrics(url_chanson):
    """récupère les paroles d'une chanson pour un url donné"""
    get_url = requests.get(url_chanson)

    if get_url.status_code == 200:
        result_html_chanson = soup(get_url.text, 'html.parser')

        paroles_list = result_html_chanson.find(
            'div', {'class': 'song-text'}).findAll('div')
        paroles_total = ""
        for paroles in paroles_list:
            paroles_total = paroles_total + paroles.text

        # paroles_total_process = paroles_total.replace(
        #     "\n", " ").replace("\r", " ").strip()
        paroles_total_process = paroles_total.replace(
            "\n", " ").replace("\r", ".").strip()
        paroles_total_process = re.sub("\s{2,}", " ", paroles_total_process)
        # TODO : ne remplacer que les espaces et non les \r
        return paroles_total_process
    else:
        print(f"ERROR pour l'url {get_url}")


def get_lyrics_artist(artist):
    """renvoie un data.frame pour un artiste donné """
    # print(artist)
    liste_chansons = get_list_song_artist(artist)
    if liste_chansons is not None:
        _ = [chanson.update({"lyrics": get_lyrics(chanson['url_lyrics'])})
             for chanson in liste_chansons]
        return pd.DataFrame.from_records(liste_chansons)
    else:
        print(f"Erreur pour l'artist {artist}")
        return None


In [4]:
def process_artist(artist):
    artist = artist.lower()
    # Enlever les lettres à accent
    artist = unidecode.unidecode(artist)
    artist = re.sub("('|&|_|\.|!)", "", artist)
    # enlever les multiples espaces
    artist = re.sub("\s{2,}", " ", artist)
    artist = artist.replace(" ", "-")
    return artist


In [5]:
df_paroles = get_list_song_artist('france-gall')

In [6]:
df_paroles

[{'artist': 'france-gall',
  'title': 'Aime-la',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-aime-la'},
 {'artist': 'france-gall',
  'title': 'Amor tambien',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-amor-tambien'},
 {'artist': 'france-gall',
  'title': 'Babacar',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-babacar'},
 {'artist': 'france-gall',
  'title': 'Bébé comme la vie',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-bebe-comme-la-vie'},
 {'artist': 'france-gall',
  'title': "C'est Bon Que Tu Sois Là",
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-c-est-bon-que-tu-sois-la'},
 {'artist': 'france-gall',
  'title': 'Calypso',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-calypso'},
 {'artist': 'france-gall',
  'title': 'Ce soir je ne dors pas',
  'url_lyrics': 'https://www.paroles.net/france-gall/paroles-ce-soir-je-ne-dors-pas'},
 {'artist': 'france-gall',
  'title': 'Cézanne peint',
  'u

#### API DEEZER

In [7]:
def get_deezer_playlist(playlist_id):
    url = f"https://api.deezer.com/playlist/{playlist_id}"
    result_api = requests.get(url)
    result_api = json.loads(result_api.text)
    playlist = []
    for track in result_api['tracks']['data']:
        detail_track = {"track_id": track['id'],
                        "track_title": track['title'],
                        "artist_id": track['artist']['id'],
                        "artist": track['artist']['name'],
                        "album_id": track['album']['id'],
                        "album": track['album']['title']}
        playlist.append(detail_track)
    return pd.DataFrame(playlist)


________________________________

### Chargement des données des playlists Deezer 

In [9]:
playlist_deezer = pd.read_csv("playlists_deezer.csv")
playlist_deezer.head()

,palylist_id,title
0,791349661,Bleu blanc hits 80
1,1420459465,Essentiels variété française
2,700895155,Essentiels chanson française
3,1189520191,Bleu blanc hits
4,764714001,Cocoripop


In [11]:
# Boucler sur une liste d'id de playlist
start = datetime.datetime.now()
list_all_playlist = [get_deezer_playlist(id) for id in playlist_deezer['palylist_id']]
df_all_playlist = pd.concat(list_all_playlist, axis=0)
end = datetime.datetime.now()
print(f"execution time : {end-start}")


execution time : 0:00:06.651364


In [14]:
print("shape : ", df_all_playlist.shape)
print("len artisit unique : ", len(df_all_playlist.artist.unique()))

shape :  (2606, 6)
len artisit unique :  890


In [15]:
df_all_playlist['artist_process'] = df_all_playlist['artist'].apply(process_artist)

artistes_to_query = df_all_playlist['artist_process'].unique()

print(len(artistes_to_query))


888


In [17]:
df_all_playlist

,track_id,track_title,artist_id,artist,album_id,album,artist_process
0,46251471,"Ella, elle l'a (Remasterisé en 2004)",4675,France Gall,4578131,Babacar (Remasterisé en 2004),france-gall
1,5151905,Mistral gagnant,21,Renaud,472854,Mistral gagnant,renaud
2,118584880,Mon fils ma bataille,26,Daniel Balavoine,12330296,Un autre monde,daniel-balavoine
3,7120489,Quand la musique est bonne,1120,Jean-Jacques Goldman,71644,Quand la musique est bonne,jean-jacques-goldman
4,2308590,Je te promets,1060,Johnny Hallyday,230008,Gang,johnny-hallyday
...,...,...,...,...,...,...,...
395,2150060,O C B,672,Billy Ze Kick et Les Gamins En Folie,215350,Billy Ze Kick Et Les Gamins En Folie,billy-ze-kick-et-les-gamins-en-folie
396,1244930842,Petite Marie (Fanzine Remix),1377312,Biga Ranx,207835122,Petite Marie (Fanzine Remix),biga-ranx
397,1187935552,Ça ira mieux demain,4791064,Grezou,195353002,"Les vibes de l'année, Vol. 2",grezou
398,1187935532,Le soleil brille encore,4791064,Grezou,195353002,"Les vibes de l'année, Vol. 2",grezou


In [18]:
artistes_to_query

array(['france-gall', 'renaud', 'daniel-balavoine',
       'jean-jacques-goldman', 'johnny-hallyday', 'marc-lavoine',
       'michel-berger', 'patrick-bruel', 'indochine', 'telephone',
       'francis-cabrel', 'gold', 'gilbert-montagne',
       'partenaire-particulier', 'jeanne-mas', 'mylene-farmer', 'images',
       'les-rita-mitsouko', 'elli-jacno', 'vanessa-paradis',
       'eddy-mitchell', 'alain-bashung', 'claude-nougaro',
       'jean-louis-aubert', 'florent-pagny', 'maxime-le-forestier',
       'michel-sardou', 'rose-laurens', 'herve-cristiani',
       'stephan-eicher', 'debut-de-soiree', 'julie-pietri',
       'thierry-pastor', 'desireless', 'peter-sloane', 'pierre-bachelet',
       'gerard-blanc', 'les-enfoires', 'julien-clerc', 'francois-feldman',
       'calogero', 'renan-luce', 'vitaa', 'gregory-lemarchal',
       'christophe-mae', 'celine-dion', 'garou', 'gregoire', 'm-pokora',
       'daniel-levi', 'louise-attaque', 'claudio-capeo', 'axelle-red',
       'de-palmas', 'emma

In [19]:
start = datetime.datetime.now()
list_df_lyrics = [get_lyrics_artist(artist) for artist in artistes_to_query]
end = datetime.datetime.now()
print(f"-- execution time : {end-start}")
df_total = pd.concat(list_df_lyrics)
print(df_total.shape)
print(df_total.artist.value_counts())


elli-jacno - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist elli-jacno
stephan-eicher - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist stephan-eicher
julien-clerc - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist julien-clerc
mentissa - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist mentissa
zaoui - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist zaoui
fils-cara - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist fils-cara
alice-et-moi - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist alice-et-moi
styleto - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist styleto
bro - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist bro
saskia - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist saskia
oordaya - Erreur dans le scraping de la liste des chansons
Erreur pour l'artist oordaya


In [20]:
df_total

,artist,title,url_lyrics,lyrics
0,france-gall,Aime-la,https://www.paroles.net/france-gall/paroles-ai...,". La chance a tourné dans ta vie, tes amours. ..."
1,france-gall,Amor tambien,https://www.paroles.net/france-gall/paroles-am...,Y a des jours fragiles. Des soirées difficiles...
2,france-gall,Babacar,https://www.paroles.net/france-gall/paroles-ba...,J'ai ton cœur qui tape qui cogne. Dans mon cor...
3,france-gall,Bébé comme la vie,https://www.paroles.net/france-gall/paroles-be...,"Y a des couleurs qu'on oublie pas. Le cœur, qu..."
4,france-gall,C'est Bon Que Tu Sois Là,https://www.paroles.net/france-gall/paroles-c-...,Tous ces mots qui condamnent. Comme c'est dur ...
...,...,...,...,...
21,dub-silence,L'hymne des légumes,https://www.paroles.net/dub-silence/paroles-l-...,"Mon style est vrai, j’ai le flow, toujours fra..."
22,dub-silence,Lovely Friends,https://www.paroles.net/dub-silence/paroles-lo...,"I am nobody, nobody, nobody,. Without my frien..."
23,dub-silence,MAJ,https://www.paroles.net/dub-silence/paroles-maj,Il y a un petit bout de temps que je vous ai r...
24,dub-silence,Matchgirl,https://www.paroles.net/dub-silence/paroles-ma...,"Demain c'est le 25 décembre, la ville s'est pa..."


In [21]:
df_paroles_artist_id = df_all_playlist[['artist', 'artist_id', 'artist_process']].drop_duplicates().merge(df_total, left_on = ['artist_process'], right_on = ['artist'])

In [22]:
df_paroles_artist_id

,artist_x,artist_id,artist_process,artist_y,title,url_lyrics,lyrics
0,France Gall,4675,france-gall,france-gall,Aime-la,https://www.paroles.net/france-gall/paroles-ai...,". La chance a tourné dans ta vie, tes amours. ..."
1,France Gall,4675,france-gall,france-gall,Amor tambien,https://www.paroles.net/france-gall/paroles-am...,Y a des jours fragiles. Des soirées difficiles...
2,France Gall,4675,france-gall,france-gall,Babacar,https://www.paroles.net/france-gall/paroles-ba...,J'ai ton cœur qui tape qui cogne. Dans mon cor...
3,France Gall,4675,france-gall,france-gall,Bébé comme la vie,https://www.paroles.net/france-gall/paroles-be...,"Y a des couleurs qu'on oublie pas. Le cœur, qu..."
4,France Gall,4675,france-gall,france-gall,C'est Bon Que Tu Sois Là,https://www.paroles.net/france-gall/paroles-c-...,Tous ces mots qui condamnent. Comme c'est dur ...
...,...,...,...,...,...,...,...
26563,Dub Silence,1585174,dub-silence,dub-silence,L'hymne des légumes,https://www.paroles.net/dub-silence/paroles-l-...,"Mon style est vrai, j’ai le flow, toujours fra..."
26564,Dub Silence,1585174,dub-silence,dub-silence,Lovely Friends,https://www.paroles.net/dub-silence/paroles-lo...,"I am nobody, nobody, nobody,. Without my frien..."
26565,Dub Silence,1585174,dub-silence,dub-silence,MAJ,https://www.paroles.net/dub-silence/paroles-maj,Il y a un petit bout de temps que je vous ai r...
26566,Dub Silence,1585174,dub-silence,dub-silence,Matchgirl,https://www.paroles.net/dub-silence/paroles-ma...,"Demain c'est le 25 décembre, la ville s'est pa..."


In [85]:
df_paroles_artist_id.to_csv("paroles_artist_id_v2.csv", sep = ";", index=False)

## Genre

In [24]:
def get_deezer_genre(album_id):
    url_album = f"https://api.deezer.com/album/{album_id}"
    result_album = requests.get(url_album)
    if result_album.status_code == 200:
        result_album = json.loads(result_album.text)
        if "error" in result_album:
            return {"artist_id": None,
                    "album_id": None,
                    "genres": None}
        else:
            detail_album = {"artist_id": result_album['artist']['id'],
                            "album_id": result_album['id'],
                            "genres": [g["name"] for g in result_album['genres']['data']]}
            return(detail_album)
    else:
        return None

In [25]:
def genre_majoritaire(liste_id_playlist) :
    list_all_playlist = [get_deezer_playlist(id) for id in liste_id_playlist]
    df_all_playlist = pd.concat(list_all_playlist, axis=0)
    df_all_playlist_genre = df_all_playlist['album_id'].apply(get_deezer_genre)
    df_all_playlist_genre = pd.DataFrame.from_records(list(df_all_playlist_genre))
    df_all_playlist_genre = df_all_playlist_genre[~df_all_playlist_genre['album_id'].isna()]

    df_all_playlist_genre_explode = df_all_playlist_genre.explode('genres')
    df_all_playlist_genre_explode = df_all_playlist_genre_explode.drop_duplicates()
    dict = {
        'Pop internationale' : 'Pop',
        'Pop indé/Folk' : 'Pop Indé',
        'Rock indé' : 'Rock',
        'Hard Rock' : 'Rock',
        'Rock Indé/Pop Rock' : 'Rock',
        'Dance' : 'Electro',
        'Electro Pop/Electro Rock' : 'Electro',
        'Techno/House' : 'Electro',
        'Films/Jeux vidéo' : 'Bandes originales',
        'Comédies musicales' : 'Bandes originales',
        'Musiques de films' : 'Bandes originales',
        'Rap/Hip Hop' : 'Rap',
        'Rap français' : 'Rap',
        'Metal' : 'Rock',
        'Rock français' : 'Rock',
        'Dub' : 'Reggae',
        'Ska' : 'Reggae',
        'Dancehall/Ragga' : 'Reggae',
        'Soul & Funk' : 'Soul',
        'R&B contemporain' : 'R&B'
    }
    df_all_playlist_genre_explode = df_all_playlist_genre_explode.replace({"genres" : dict})
    df_all_playlist_genre_explode = pd.concat([df_all_playlist_genre_explode, pd.get_dummies(df_all_playlist_genre_explode['genres'])], axis=1)
    df_all_playlist_genre_explode = df_all_playlist_genre_explode[~df_all_playlist_genre_explode.genres.isna()]

    unique_genres = df_all_playlist_genre_explode['genres'].unique()
    genres_by_artist = df_all_playlist_genre_explode.groupby(['artist_id'])[unique_genres].sum()
    genres_by_artist['GENRES_MAX'] = genres_by_artist.idxmax(axis=1)
    genres_by_artist['GENRES_MAX_VALUE'] = genres_by_artist[unique_genres].max(axis=1)
    genres_by_artist = genres_by_artist.reset_index()

    genres_by_artist['GENRES_SUM_ROW'] = genres_by_artist[unique_genres].sum(axis=1)
    genres_by_artist['MOYENNE_ROW'] = genres_by_artist[unique_genres].replace({0: np.nan})[unique_genres].mean(axis=1)
    genres_by_artist['MAX_CERTAINTY'] = genres_by_artist.apply(lambda x :  not x['GENRES_MAX_VALUE'] != x['MOYENNE_ROW'] , axis=1)
    genres_by_artist['artist_id'] = genres_by_artist['artist_id'].astype(int)

    # return(df_all_playlist, genres_by_artist[["GENRES_MAX", "MAX_CERTAINTY"]].reset_index())
    return(genres_by_artist)




In [27]:
pd.set_option('display.max_columns', None)
liste_id_playlist = pd.read_csv("playlists_deezer.csv")
liste_id_playlist = liste_id_playlist['palylist_id']
genres_by_artist = genre_majoritaire(liste_id_playlist)
genres_by_artist

,artist_id,Pop,Chanson française,Rock,Variété Internationale,Alternative,Disco,Soul,Rap,Electro,Jazz,Pop Indé,Singer & Songwriter,R&B,Bandes originales,Folk,Classique,Latino,Musique africaine,Reggae,Trance,Chill Out/Trip-Hop/Lounge,Moderne,Jeunesse,GENRES_MAX,GENRES_MAX_VALUE,GENRES_SUM_ROW,MOYENNE_ROW,MAX_CERTAINTY
0,18,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rap,1,1,1.0,True
1,21,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,7,7,7.0,True
2,26,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,4,4,4.0,True
3,29,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,5,5,5.0,True
4,44,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Chanson française,2,2,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,193665917,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rap,1,1,1.0,True
749,196537497,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rock,4,4,4.0,True
750,198270207,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Rock,1,2,1.0,True
751,200448247,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Pop,1,1,1.0,True


In [91]:
genres_by_artist.to_csv("genres_by_artist_v2.csv", sep = ";", index=False)

In [34]:
df_paroles_genres = pd.merge(df_paroles_artist_id, genres_by_artist[['artist_id','GENRES_MAX','MAX_CERTAINTY']], on="artist_id", how="left")
df_paroles_genres.drop(["artist_y"], axis=1, inplace=True)
df_paroles_genres = df_paroles_genres.rename(columns={"artist_x": "artist"})
df_paroles_genres

,artist,artist_id,artist_process,title,url_lyrics,lyrics,GENRES_MAX,MAX_CERTAINTY
0,France Gall,4675,france-gall,Aime-la,https://www.paroles.net/france-gall/paroles-ai...,". La chance a tourné dans ta vie, tes amours. ...",Pop,True
1,France Gall,4675,france-gall,Amor tambien,https://www.paroles.net/france-gall/paroles-am...,Y a des jours fragiles. Des soirées difficiles...,Pop,True
2,France Gall,4675,france-gall,Babacar,https://www.paroles.net/france-gall/paroles-ba...,J'ai ton cœur qui tape qui cogne. Dans mon cor...,Pop,True
3,France Gall,4675,france-gall,Bébé comme la vie,https://www.paroles.net/france-gall/paroles-be...,"Y a des couleurs qu'on oublie pas. Le cœur, qu...",Pop,True
4,France Gall,4675,france-gall,C'est Bon Que Tu Sois Là,https://www.paroles.net/france-gall/paroles-c-...,Tous ces mots qui condamnent. Comme c'est dur ...,Pop,True
...,...,...,...,...,...,...,...,...
26563,Dub Silence,1585174,dub-silence,L'hymne des légumes,https://www.paroles.net/dub-silence/paroles-l-...,"Mon style est vrai, j’ai le flow, toujours fra...",Pop,False
26564,Dub Silence,1585174,dub-silence,Lovely Friends,https://www.paroles.net/dub-silence/paroles-lo...,"I am nobody, nobody, nobody,. Without my frien...",Pop,False
26565,Dub Silence,1585174,dub-silence,MAJ,https://www.paroles.net/dub-silence/paroles-maj,Il y a un petit bout de temps que je vous ai r...,Pop,False
26566,Dub Silence,1585174,dub-silence,Matchgirl,https://www.paroles.net/dub-silence/paroles-ma...,"Demain c'est le 25 décembre, la ville s'est pa...",Pop,False


In [42]:
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang

df_paroles_genres['detect'] = df_paroles_genres['lyrics'].apply(det)
df_paroles_genres['detect'].value_counts()


In [47]:
df_paroles_genres = df_paroles_genres[df_paroles_genres['detect'] == 'fr']

In [48]:
df_paroles_genres.GENRES_MAX.value_counts()

Pop                    8012
Rap                    3859
Chanson française      2292
Rock                   2035
Reggae                  955
Alternative             354
Jazz                    239
R&B                     232
Latino                  178
Soul                    121
Musique africaine       114
Electro                 108
Singer & Songwriter      17
Bandes originales        13
Name: GENRES_MAX, dtype: int64

In [49]:
nb_chansons_artist = df_paroles_genres.groupby(['artist_id','artist','GENRES_MAX'])['lyrics'].count().sort_values(axis=0, ascending = False)
nb_chansons_artist

artist_id  artist                    GENRES_MAX         
55538      Juliette Gréco            Pop                    204
4195       Serge Reggiani            Pop                    202
1500       Pigalle                   Pop                    187
3047       Mass Hysteria             Rock                   173
3931       Jeanne Moreau             Chanson française      171
12308342   Youv Dee                  Rock                   166
1751       Marcel et son Orchestre   Chanson française      165
1191615    Jul                       Rap                    124
1261       Les Sales Majestés        Rock                   123
430        Tiken Jah Fakoly          Reggae                 120
5251       Pierpoljak                Reggae                 117
4356       Albin De La Simone        Pop                    112
183727     Georgio                   Rap                    109
898        Cali                      Pop                    108
3192       Maxime Le Forestier       Pop       

In [54]:
df_paroles_genres.to_csv("data_lyrics.csv")